In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

#%matplotlib inline

# Leo csv

In [2]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [3]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [4]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [5]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Borro duplicados y agrego edad

In [6]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [7]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

# En un principio los relleno con la media de los postulantes
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int32')
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad                   0
dtype: int64

In [8]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [9]:
postulaciones = postulaciones.drop_duplicates(keep='last')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Separo gente postulada y no postulada

In [10]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [11]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [12]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [13]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [14]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
#gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [15]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [16]:
gente_postulada.isnull().sum()

idpostulante           0
fechanacimiento    10074
sexo                   0
edad                   0
dtype: int64

# Arreglo el df de avisos detalle

In [17]:
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL
1,9240880,1,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,Capital Federal,Buenos Aires,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga
2,1110185164,1,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Administración,Unión Personal
3,1110513885,1,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Distribución,Asoko Tempo SA
4,1111034024,1,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,JELS SRL


In [ ]:
avisos_detalle['nombre_area'].unique().shape

In [18]:
avisos_detalle['nombre_area'].unique()

array(['Producción', 'Comercial', 'Administración', 'Distribución',
       'Liderazgo de Proyecto', 'Construcción', 'Call Center',
       'Recursos Humanos', 'Almacén / Depósito / Expedición', 'Ventas',
       'Soporte Técnico', 'Calidad', 'Atención al Cliente', 'Selección',
       'Comunicacion', 'Administración de Personal',
       'Planeamiento comercial', 'Gastronomia', 'Tecnico de Seguros',
       'Asistente', 'Contabilidad', 'Facturación', 'Telemarketing',
       'Créditos y Cobranzas', 'Planeamiento económico-financiero',
       'Jóvenes Profesionales', 'Capacitación', 'Secretaria',
       'Ingeniería  Eléctrica y Electrónica', 'Compras',
       'Mineria/Petroleo/Gas', 'Compras Internacionales/Importación',
       'Responsabilidad Social', 'Ingeniería  Industrial', 'Caja',
       'Auditoría', 'Seguridad', 'Ventas Internacionales/Exportación',
       'Dirección de Obra', 'Análisis Funcional',
       'Ingeniería Electromecánica', 'Desarrollo de Negocios',
       'Telecomunicacione

In [19]:
replacements = {
   'nombre_area': {
      'Ingeniería Geológica|Ingeniería Metalúrgica|Ingeniería  Metalurgica|Ingeniería  Eléctrica y Electrónica|Ingeniería  Automotriz|Ingeniería de Producto|Ingeniería de Procesos|Ingeniería de Ventas|Ingeniería Oficina Técnica / Proyecto|Ingeniería en Alimentos|Ingeniería Agrónoma|Ingeniería Química|Ingeniería  Industrial|Ingeniería Electromecánica|Ingeniería  Mecánica|Otras Ingenierias': 'Ingenieria',      
      'Corporate Finance / Banca Inversión|Evaluación Económica|Organización y Métodos|Cuentas Corrientes|Cadetería|Contabilidad|Impuestos|Auditoría|Control de Gestión|Consultoria': 'Administración',#cuentasc podria ir en ventas
      'Diseño Multimedia|Diseño 3D|Diseño Textil e Indumentaria|Diseño Web|Diseño de Interiores / Decoración|Diseño Industrial|Diseño Gráfico': 'Diseño y Urbanismo',
      'Periodismo|Comunicaciones Internas|Relaciones Institucionales/Publicas|Responsabilidad Social': 'Comunicacion',
      'Auditoría de Seguros|Venta de Seguros|Tecnico de Seguros|Administración de Seguros|Administracion de Seguros': 'Seguros',
      'Quimica|Bioquimica|Bioquímica': 'Química',
      'Selección|Capacitación|Compensación y Planilla|Administración de Personal': 'Recursos Humanos', #administracion de personal dudo
      'Exploración Minera y Petroquimica|Ingeniería en Petróleo y Petroquímica|Salud Minera|Seguridad Industrial|Medio Ambiente|Instrumentación Minera|Ingeniería en Minas|Mineria/Petroleo/Gas': 'Minería',
      'Traduccion': 'Idiomas',
      'Mantenimiento|Calidad|Programación de producción': 'Producción',
      'Inversiones / Proyectos de Inversión|Finanzas Internacionales|Análisis de Riesgos|Tesorería|Créditos y Cobranzas|Facturación|Planeamiento económico-financiero': 'Finanzas',
      'Telefonista|Call Center|Telemarketing|Caja': 'Atención al Cliente', #no se si caja va aca
      'Almacén / Depósito / Expedición|Compras|Distribución|Abastecimiento' : 'Logística',
      'Legal|Asesoría Legal Internacional': 'Legales',
      'Sistemas|Telecomunicaciones|Tecnologia/Sistemas de producción|Oficios Informática|Tecnologías de la Información|Data Warehousing|Redes|Data Entry|Administración de Base de Datos|Seguridad Informática|Soporte Técnico|Testing / QA / QC|Internet|Infraestructura|Análisis Funcional|Liderazgo de Proyecto|Programación' : 'Tecnologia/Sistemas',
      'Oficios y Profesiones|Producción y Limpieza|Estética y Cuidado Personal|Mantenimiento y Limpieza|Seguridad|Servicios|Promotoras/es|Jóvenes Profesionales': 'Oficios',#aca tambien
      'Recepcionista|Asistente': 'Secretaria',
      'Dirección de Obra|Arquitectura|Ingeniería Civil|Urbanismo|Operaciones|Diseño y Construcción|Topografía' : 'Construcción',
      'Compras Internacionales/Importación|Ventas Internacionales/Exportación|Consultorías Comercio Exterior|Apoderado Aduanal|Logística Internacionales/Importación': 'Comercio Exterior',
      'Business Intelligence|E-commerce|Community Management|Producto|Media Planning|Mercadotecnia Internacional': 'Marketing',
      'Camareros|Hotelería|Turismo': 'Gastronomia',
      'Seguridad Industrial|Oficios e Higiene' : 'Seguridad e Higiene',
      'Comercial|Ventas económico-financiero|Planeamiento Comercial|Planeamiento|Desarrollo de Negocios|Negocios Internacionales|Ventas comercial' : 'Ventas',
      'Laboratorio|Farmacia industrial|Farmacia comercial|Medicina|Otras Especialidades médicas|Diagnóstico por imágenes|Esterilización|Maternidad|Farmacéutica|Veterinaria|Odontología|Farmacia hospitalaria|Farmacia Industrial|Farmacia Comercial|Medicina Laboral|Instrumentación|Auditoría Médica|Emergentología|Kinesiología / terapia ocupacional|Prácticas cardiológicas|Otras áreas técnicas en salud|Fonoaudiología|Clínica Médica|Cuidados Intensivos Cardiologicos Adultos|Psicología|Instrumentación quirúrgica|Administración Médica' : 'Salud',
      'Gerencia / Dirección General|Dirección': 'Gerencia',
      'Pasantía / Trainee|Prácticas Profesionales': 'Pasantía',
      'Tecnologia / Tecnologia/Sistemas': 'Tecnologia/Sistemas',
      'Diseño y Construcción':'Construcción',
      'Secretaria de Tráfico': 'Logística',
      'Ventas comercial':'Ventas',
      'Salud Laboral|Salud quirúrgica':'Salud',
      'Educación/ Docentes|Educación especial|Trabajo social|Trabajo Social|Investigación y Desarrollo|Bienestar Estudiantil': 'Educación',
      'Arte y Cultura|Comunicaciones Externas|Independientes|Transporte|Creatividad|Multimedia|Back Office': 'Otros' #transporte entraria tanto en logistica como en oficios y creatividad en marketing y diseño
       #Multimedia en marketing y diseño. Back Office puede entrar en muchos
   }
}

avisos_detalle.replace(replacements, regex=True, inplace=True)

In [20]:
print(avisos_detalle['nombre_area'].value_counts())

Ventas                         5256
Administración                 3218
Tecnologia/Sistemas            3062
Producción                     2375
Atención al Cliente            1354
Logística                      1332
Oficios                        1180
Recursos Humanos               1012
Finanzas                        929
Ingenieria                      868
Salud                           725
Gastronomia                     602
Construcción                    585
Otros                           583
Marketing                       579
Secretaria                      459
Comercio Exterior               236
Legales                         235
Minería                         124
Seguros                         106
Gerencia                        102
Comunicacion                     92
Educación                        86
Pasantía                         70
Seguridad e Higiene              49
Química                          40
Diseño                           22
Idiomas                     

In [21]:
avisos_detalle.loc[avisos_detalle['nombre_area'] =='Enfermería (ver Enfermería)', "nombre_area"] = "Salud"

# Funciones

In [22]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [23]:
def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    return df_return

In [24]:
def feature_area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

In [25]:
avisos_detalle = oneHotEncoding(avisos_detalle,'nombre_area')
avisos_detalle

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,denominacion_empresa,...,Pasantía,Producción,Química,Recursos Humanos,Salud,Secretaria,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Ventas
0,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,ELECTRO OUTLET SRL,...,0,1,0,0,0,0,0,0,0,0
1,9240880,1,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,Capital Federal,Buenos Aires,NaN,Full-time,Jefe / Supervisor / Responsable,Agencia Oficial Alejandro Arizaga,...,0,0,0,0,0,0,0,0,0,1
2,1110185164,1,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Unión Personal,...,0,0,0,0,0,0,0,0,0,0
3,1110513885,1,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Asoko Tempo SA,...,0,0,0,0,0,0,0,0,0,0
4,1111034024,1,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,JELS SRL,...,0,0,0,0,0,0,0,0,0,1
5,1111101289,1,PM. Zona Norte / Pacheco,<p>Acciona IT se encuentra en la búsqueda de P...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,ACCIONA IT,...,0,0,0,0,0,0,0,0,1,0
6,1111109704,1,"Supervisor de obra civil, eléctrico/ instrumen...",<p>Buscamos para importante empresa petrolera ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Hahn Solarz S.R.L.,...,0,0,0,0,0,0,0,0,0,0
7,1111172357,1,Empleado Administrativo para Tramites de Habil...,"<p style=""""><strong><em><span style="""">Brujula...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Brujula SA,...,0,0,0,0,0,0,0,0,0,0
8,1111174081,1,Gestor de Cobranzas - Telecobrador,<p><strong>En MAS ACTIVOS BPO te estamos esper...,Capital Federal,NaN,ALEM 116,Part-time,Senior / Semi-Senior,MAS ACTIVOS S.A.,...,0,0,0,0,0,0,0,0,0,0
9,1111235995,1,Analista de Recursos Humanos - Quilmes (Plazo...,<p>Cervecería y Maltería Quilmes es una de las...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,AB InBev - Cervecería y Maltería Quilmes,...,0,0,0,1,0,0,0,0,0,0


# Test final

In [26]:
test_final = feature_genero_edad(test_final,gente_postulada.append(gente_no_postulada))
test_final

,idaviso,idpostulante,fechanacimiento,sexo,edad
0,739260,6M9ZQR,1976-03-29,FEM,42.0
1,739260,6v1xdL,1987-06-27,MASC,31.0
2,739260,ezRKm9,1982-02-17,FEM,36.0
3,758580,1Q35ej,1949-12-24,MASC,69.0
4,758580,EAN4J6,1986-05-27,FEM,32.0
5,758580,8R6pzR,1990-05-29,MASC,28.0
6,776420,aZJ2XN,1989-01-23,MASC,29.0
7,776420,Nmpo3J,1964-02-13,MASC,54.0
8,776420,eVqWar,1990-04-09,FEM,28.0
9,820850,6ZBD33,1980-09-18,MASC,38.0


In [27]:
test_final = feature_area_de_trabajo(test_final)

In [28]:
test_final

,idaviso,idpostulante,fechanacimiento,sexo,edad,idpais,titulo,descripcion,nombre_zona,ciudad,...,Pasantía,Producción,Química,Recursos Humanos,Salud,Secretaria,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Ventas
0,739260,6M9ZQR,1976-03-29,FEM,42.0,1,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,...,0,0,0,0,0,0,0,0,0,1
1,739260,6v1xdL,1987-06-27,MASC,31.0,1,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,...,0,0,0,0,0,0,0,0,0,1
2,739260,ezRKm9,1982-02-17,FEM,36.0,1,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,...,0,0,0,0,0,0,0,0,0,1
3,758580,1Q35ej,1949-12-24,MASC,69.0,1,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,NaN,...,0,0,0,0,0,0,0,0,1,0
4,758580,EAN4J6,1986-05-27,FEM,32.0,1,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,NaN,...,0,0,0,0,0,0,0,0,1,0
5,758580,8R6pzR,1990-05-29,MASC,28.0,1,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,NaN,...,0,0,0,0,0,0,0,0,1,0
6,776420,aZJ2XN,1989-01-23,MASC,29.0,1,Consultores Senior y SemiSr de Procesos,"Somos una Consultora en Management, Transforma...",Capital Federal,NaN,...,0,0,0,0,0,0,0,0,0,0
7,776420,Nmpo3J,1964-02-13,MASC,54.0,1,Consultores Senior y SemiSr de Procesos,"Somos una Consultora en Management, Transforma...",Capital Federal,NaN,...,0,0,0,0,0,0,0,0,0,0
8,776420,eVqWar,1990-04-09,FEM,28.0,1,Consultores Senior y SemiSr de Procesos,"Somos una Consultora en Management, Transforma...",Capital Federal,NaN,...,0,0,0,0,0,0,0,0,0,0
9,820850,6ZBD33,1980-09-18,MASC,38.0,1,Administrativo de RRHH,Importante compañía de packaging interesada en...,Buenos Aires (fuera de GBA),NaN,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
test_final.isnull().sum()

In [29]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isnull(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [30]:
test_final = test_final.drop(['idaviso','idpostulante','fechanacimiento','ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)
test_final['edad'] = test_final['edad'].astype('int32')
test_final = oneHotEncoding(test_final,'sexo')

In [31]:
test_final.isnull().sum()

edad                   0
Administración         0
Atención al Cliente    0
Comercio Exterior      0
Comunicacion           0
Construcción           0
Diseño                 0
Educación              0
Finanzas               0
Gastronomia            0
Gerencia               0
Idiomas                0
Ingenieria             0
Legales                0
Logística              0
Marketing              0
Minería                0
Oficios                0
Otros                  0
Pasantía               0
Producción             0
Química                0
Recursos Humanos       0
Salud                  0
Secretaria             0
Seguridad e Higiene    0
Seguros                0
Tecnologia/Sistemas    0
Ventas                 0
FEM                    0
MASC                   0
NO_DECLARA             0
dtype: int64

In [32]:
test_final

,edad,Administración,Atención al Cliente,Comercio Exterior,Comunicacion,Construcción,Diseño,Educación,Finanzas,Gastronomia,...,Recursos Humanos,Salud,Secretaria,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Ventas,FEM,MASC,NO_DECLARA
0,42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,36,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,69,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
5,28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
6,29,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,54,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,28,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,38,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


# Funciones utilidad

In [33]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [34]:
# fijarse de balancear el sample con 50% de postulados y no postulados

# Generador de sample
# el id tiene que estar en la columna 0
# Recibe los DFs de idpostulante e idaviso
def generador_de_sample_df(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = idpostulante.drop(columns=idpostulante.columns[1:])
    idaviso = idaviso.drop(columns=idaviso.columns[1:])
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

# Tupla postulada y sample no postulado

In [37]:
tupla_postulada = postulaciones.drop('fechapostulacion',axis=1)
tupla_postulada['y'] = 1
tupla_postulada = feature_genero_edad(tupla_postulada,gente_postulada)
tupla_postulada = feature_area_de_trabajo(tupla_postulada)
tupla_postulada = tupla_postulada.drop(['fechanacimiento','ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')

MemoryError: 

In [ ]:
tupla_postulada.head()

In [35]:
sample = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],100,100)
sample['y'] = 0
sample = feature_genero_edad(sample,gente_no_postulada)
sample = feature_area_de_trabajo(sample)
sample = sample.drop(['fechanacimiento','ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)
sample = oneHotEncoding(sample,'sexo')

In [36]:
sample.head()

,idaviso,idpostulante,y,edad,Administración,Atención al Cliente,Comercio Exterior,Comunicacion,Construcción,Diseño,...,Recursos Humanos,Salud,Secretaria,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Ventas,FEM,MASC,NO_DECLARA
0,1112390041,GNZ8JWK,0,55,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1112390041,ZDlz5E1,0,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1112390041,a0N4q,0,43,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1112390041,ep41j9,0,32,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1112390041,awOL1m,0,34,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Borro lo que no necesito

In [ ]:
del avisos_detalle
del postulantes_genero_y_edad
del postulaciones

# Separo la edad por rango en el sample y el test

In [ ]:
sample = sample.append(tupla_postulada.sample(n=1000000,random_state=0))

In [ ]:
sample = sample.drop(['idaviso','idpostulante'],axis=1)

In [ ]:
sample['edad'] = pd.cut(sample['edad'], range(0, 100, 5))
sample['edad'] = sample['edad'].astype(str)
dic = {'(10, 15]':0 , '(15, 20]':1 , '(20, 25]':2 , '(25, 30]':3 , '(30, 35]':4 , '(35, 40]':5 , '(40, 45]':6 , '(45, 50]':7 , '(50, 55]':8 , '(55, 60]':9 , '(60, 65]':10 , '(65, 70]':11 , '(70, 75]':12 , '(75, 80]':13 , '(80, 85]':14 , '(85, 90]':15 , '(95, 100]':16 , '(240, 245]':17}
sample['edad'] = sample['edad'].map(dic)

In [ ]:
test_final['edad'] = pd.cut(test_final['edad'], range(0, 100, 5))
test_final['edad'] = test_final['edad'].astype(str)
test_final['edad'] = test_final['edad'].map(dic)

In [ ]:
sample['edad'] = sample['edad'].fillna(2)

In [ ]:
test_final['edad'] = test_final['edad'].fillna(2)

In [ ]:
test_final.isnull().sum()

# Random Forest

In [ ]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_label = []
train_data = []

train_label = sample.iloc[:, 0]
train_data = sample.iloc[:, sample.columns != 'y']

In [ ]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [ ]:
ModeloRFC.fit(train_data, train_label)

In [ ]:
result = ModeloRFC.predict(test_final)

In [ ]:
result.size

In [ ]:
import csv

In [ ]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'sepostulo'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [ ]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/RFC_edadPorRango.csv")